In [1]:
import os
import gc
import numpy as np
import pandas as pd
from tqdm import tqdm
from typing import Dict, List, Tuple, NamedTuple, Callable
import scml
import mylib

In [2]:
percentiles=[.01, .05, .1, .2, .3, .4, .5, .6, .7, .8, .9, .95, .99]
os.environ["TOKENIZERS_PARALLELISM"] = "false"
pd.set_option("use_inf_as_na", True)
pd.set_option("max_info_columns", 9999)
pd.set_option("display.max_columns", 9999)
pd.set_option("display.max_rows", 9999)
pd.set_option('max_colwidth', 9999)
tqdm.pandas()

In [3]:
%%time
df = pd.read_csv("input/validation_data.csv", engine="c", low_memory=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30108 entries, 0 to 30107
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   worker      30108 non-null  int64 
 1   less_toxic  30108 non-null  object
 2   more_toxic  30108 non-null  object
dtypes: int64(1), object(2)
memory usage: 705.8+ KB
Wall time: 201 ms


In [4]:
texts = set(df["less_toxic"]) | set(df["more_toxic"])
texts = list(texts)
df = pd.DataFrame(data={"text": texts})
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14251 entries, 0 to 14250
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    14251 non-null  object
dtypes: object(1)
memory usage: 111.5+ KB


# Preprocess Text
Speed dropped from 1200 to 1000 it/s

In [5]:
def pre1(row) -> str:
    return mylib.pre1(row["text"])


def pre2(row) -> str:
    return mylib.pre2(row["text1"])


col = "text1"
print(col)
df[col] = df.progress_apply(pre1, axis=1)
col = "text2"
print(col)
df[col] = df.progress_apply(pre2, axis=1)

text1


100%|████████████████████████████████████████| 14251/14251 [00:45<00:00, 311.75it/s]


text2


100%|█████████████████████████████████████████| 14251/14251 [04:16<00:00, 55.50it/s]


In [6]:
cols = ["text", "text1", "text2"]
df[cols].sample(10).head(10)

,text,text1,text2
12982,"Catholic Jihad against Evangelicals \n\nI want to post,write articles on masssacres,persecution,terrorism and repression by Catholics against baptists and pentecostals. Specially on baptist holocaust in Colombia(1948-1960). But you a Baptist,delete any reference to any wrong doing by catholics. What it can be made in relation to it.\nWe Latin American Baptists had human rights,we had right to denounce crimes and atrocities comitted agaist us.\nWhy you violate our freedom of speech rights?.","Catholic Jihad against Evangelicals I want to post,write articles on masssacres,persecution,terrorism and repression by Catholics against baptists and pentecostals. Specially on baptist holocaust in Colombia (1948-1960). But you a Baptist,delete any reference to any wrong doing by catholics. What it can be made in relation to it. We Latin American Baptists had human rights,we had right to denounce crimes and atrocities comitted agaist us. Why you violate our freedom of speech rights?.","Catholic Jihad against Evangelicals I want to post,write articles on masssacres,persecution,terrorism and repression by Catholics against baptists and pentecostals. Specially on baptist holocaust in Colombia (1948-1960). But you a Baptist,delete any reference to any wrong doing by catholics. What it can be made in relation to it. We Latin American Baptists had human rights,we had right to denounce crimes and atrocities comitted agaist us. Why you violate our freedom of speech rights?."
11332,"Max, I seriously don't give a flying fuck about you, Jakezing, or your attitudes towards each other. What's done is done. What you need to do now is move on. Forget about Jake. You'll probably never see him again on Wikipedia. You've spoiled your otherwise-clean record dealing with that pompous jackass, and you don't need to do any more damage to it. So he didn't tell you why he had a problem with you. Quite frankly, I can't understand why either of you kept arguing with the other for so long. I believe that's just poor judgment, really. In any case, I don't have to help you with anything, nor will I. The only reason I stepped into this was because I was trying to stop you two from ripping each others' throats out, and that didn't do any good, since you two just kept right on. He might have confused you with someone else, but that was no reason for you to keep harassing him about it. You and him both have displayed abuse of WP:CIVIL and WP:NPA on countless occasions. Now, please, do us all a favor and go on your way. ()(talk)(evidence)","Max, I seriously don't give a flying fuck about you, Jakezing, or your attitudes towards each other. What's done is done. What you need to do now is move on. Forget about Jake. You'll probably never see him again on Wikipedia. You've spoiled your otherwise-clean record dealing with that pompous jackass, and you don't need to do any more damage to it. So he didn't tell you why he had a problem with you. Quite frankly, I can't understand why either of you kept arguing with the other for so long. I believe that's just poor judgment, really. In any case, I don't have to help you with anything, nor will I. The only reason I stepped into this was because I was trying to stop you two from ripping each others' throats out, and that didn't do any good, since you two just kept right on. He might have confused you with someone else, but that was no reason for you to keep harassing him about it. You and him both have displayed abuse of WP (CIVIL and WP) NPA on countless occasions. Now, please, do us all a favor and go on your way. () (talk) (evidence)","Max, I seriously do not give a flying fuck about you, Jakezing, or your attitudes towards each other. what is done is done. What you need to do now is move on. Forget about Jake. you will probably never see him again on Wikipedia. you have spoiled your otherwise-clean record dealing with that pompous jackass, and you do not need to do any more damage to it. So he did not te

In [7]:
df[cols].sample(10).head(10)

,text,text1,text2
13516,i said PISS OFF YOU GEEKY LITTLE SHIT,i said PISS OFF YOU GEEKY LITTLE SHIT,i said PISS OFF YOU GEEKY LITTLE SHIT
2678,"piss off, this is my talk page.","piss off, this is my talk page.","piss off, this is my talk page."
6635,Two dilettantes arguing... Wikipedia...,Two dilettantes arguing... Wikipedia...,Two dilettantes arguing... Wikipedia...
8318,"I never said we need to cover up the fact. I said that the fact is irrelevant to anal sex, and the way it is presented in the article leads to a colorful and inaccurate interpretation of male-male anal sex. It is not the most straight-forward way to explain pleasure from male receptive anal sexit is factually and logically a roundabout way to do it, and it is misleading. Really, that's what the issue is. As I have several times mentioned, the only thing that is relevant to anal sex is that prostate stimulation can be pleasureable; it's homology with the Skene's gland is completely irrelevant. Also, do you think that the psychological factor plays no role in receptive pleasure? I personally think that it plays a larger role than the prostate does, but it isn't mentioned once in the article.","I never said we need to cover up the fact. I said that the fact is irrelevant to anal sex, and the way it is presented in the article leads to a colorful and inaccurate interpretation of male-male anal sex. It is not the most straight-forward way to explain pleasure from male receptive anal sexit is factually and logically a roundabout way to do it, and it is misleading. Really, that's what the issue is. As I have several times mentioned, the only thing that is relevant to anal sex is that prostate stimulation can be pleasureable; it's homology with the Skene's gland is completely irrelevant. Also, do you think that the psychological factor plays no role in receptive pleasure? I personally think that it plays a larger role than the prostate does, but it isn't mentioned once in the article.","I never said we need to cover up the fact. I said that the fact is irrelevant to anal sex, and the way it is presented in the article leads to a colorful and inaccurate interpretation of male-male anal sex. It is not the most straight-forward way to explain pleasure from male receptive anal [to make a hasty exit while in the act of sexual intercourse] is factually and logically a roundabout way to do it, and it is misleading. Really, that is what the issue is. As I have several times mentioned, the only thing that is relevant to anal sex is that prostate stimulation can be pleasureable; it is homology with the Skene's gland is completely irrelevant. Also, do you think that the psychological factor plays no role in receptive pleasure? I personally think that it plays a larger role than the prostate does, but it is not mentioned once in the article."
7896,"""\n\nYou're both wrong. Stop this useless shouting match immediately. If you don't stop, I'll block you, and this goes for you both. - (conspire | past ops) """,""" You're both wrong. Stop this useless shouting match immediately. If you don't stop, I'll block you, and this goes for you both. - (conspire | past ops) """,""" you are both wrong. Stop this useless shouting match immediately. If you do not stop, I will block you, and this goes for you both. - (conspire | past ops) """
5175,"Falcon, you can run and hide but you will never have the balls that God handed to Mark Slater, and maybe only a tiny fraction of the brains. I don't know what you look like but I suppose that if you only have two eyes, a nose, and mouth, compared to Slater, you may be ahead of the game in the looks department. BUT...he obviously doesn't care about looks, since he's got the balls and brains of an enlightened warrior, something you will never have. You only wish I were a troll, so that you can demonize me, and consign me to a lower level of species, like some revolting gargoyle hanging off a church. I am not a troll, I am a defender of Truth, and yes, I

In [8]:
df[cols].sample(10).head(10)

,text,text1,text2
4849,"Upset \n\nI TRUSTED YOU!! IT TOOK ME LIKE, 15 MINUTES TO TYPE THAT PARAGRAPH! AND NOW YOU STILL TAKE HIS SIDE? CANT YOU SEE THAT IM THE VICTIM!! Im so lonley.","Upset I TRUSTED YOU!! IT TOOK ME LIKE, 15 MINUTES TO TYPE THAT PARAGRAPH! AND NOW YOU STILL TAKE HIS SIDE? CANT YOU SEE THAT IM THE VICTIM!! Im so lonley.","Upset I TRUSTED YOU!! IT TOOK ME LIKE, 15 MINUTES TO TYPE THAT PARAGRAPH! AND NOW YOU STILL TAKE HIS SIDE? CANT YOU SEE THAT IM THE VICTIM!! Im so lonley."
11487,Kill the Muslim suporter of palastinians/terrorists,Kill the Muslim suporter of palastinians/terrorists,Kill the Muslim suporter of palastinians/terrorists
6947,"""\n\nEventually, I foresee some of ACORN's self serving, spin doctored crap getting into the article if it's prefaced by """"David Lagstein, spokesman for ACORN's Detroit office, said ..."""" Surely you don't want their version to go unchallenged, do you? Moncrief worked for Project Vote, a closely-intertwined, symbiotic ACORN subsidiary, for three years - as long as Obama has been a senator. Call her """"fired ACORN worker Anita Moncrief,"""" put the facts into the article and let the reader decide. I'd like to review what people in my profession call """"reliability indicators."""" If she gets caught lying, she faces 5 years in prison on a perjury charge. If Lagstein gets caught lying, he faces ... no consequences whatsoever. Answer my GM analogy on the article Talk page. You've been carefully avoiding it. """,""" Eventually, I foresee some of ACORN's self serving, spin doctored crap getting into the article if it's prefaced by """"David Lagstein, spokesman for ACORN's Detroit office, said ..."""" Surely you don't want their version to go unchallenged, do you? Moncrief worked for Project Vote, a closely-intertwined, symbiotic ACORN subsidiary, for three years - as long as Obama has been a senator. Call her """"fired ACORN worker Anita Moncrief,"""" put the facts into the article and let the reader decide. I'd like to review what people in my profession call """"reliability indicators."""" If she gets caught lying, she faces 5 years in prison on a perjury charge. If Lagstein gets caught lying, he faces ... no consequences whatsoever. Answer my GM analogy on the article Talk page. You've been carefully avoiding it. """,""" Eventually, I foresee some of ACORN's self serving, spin doctored crap getting into the article if it is prefaced by """"David Lagstein, spokesman for ACORN's Detroit office, said ..."""" Surely you do not want their version to go unchallenged, do you? Moncrief worked for Project Vote, a closely-intertwined, symbiotic ACORN subsidiary, for three years - as long as Obama has been a senator. Call her """"fired ACORN worker Anita Moncrief,"""" put the facts into the article and let the reader decide. I would like to review what people in my profession call """"reliability indicators."""" If she gets caught lying, she faces 5 years in prison on a perjury charge. If Lagstein gets caught lying, he faces ... no consequences whatsoever. Answer my GM analogy on the article Talk page. you have been carefully avoiding it. """
10717,Buffalo Bill THE VANDAL\nYOU MY FRIEND ARE LITTLE MORE THAN A VANDAL AND AN EGOCENTRIC JERK\n,Buffalo Bill THE VANDAL YOU MY FRIEND ARE LITTLE MORE THAN A VANDAL AND AN EGOCENTRIC JERK,Buffalo Bill THE VANDAL YOU MY FRIEND ARE LITTLE MORE THAN A VANDAL AND AN EGOCENTRIC JERK
5980,"""\n\n Edit request on 6 February 2012 \n\nDefinition of Islamophobia\n Literal Definition of Phobia\npho·bi·a (fō'bē-ə): A persistent, abnormal, and irrational fear of a specific thing or situation that compels one to avoid it, despite the awareness and reassurance that it is not dangeroushttp://dictionary.reference.com/browse/Phobia.\n\n According to the Organisation of The Islamic Conference: \n""""Islamophobia signifies the contemporary proliferation of discrimination against Muslims and distortion of Islam and is partly due to the ignorance and lack of und

# Review data

In [9]:
df[cols].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14251 entries, 0 to 14250
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    14251 non-null  object
 1   text1   14251 non-null  object
 2   text2   14251 non-null  object
dtypes: object(3)
memory usage: 334.1+ KB


In [10]:
%%time
df[cols].to_parquet("output/pre_val.parquet", index=False)

Wall time: 115 ms
